In [79]:
import json
import pandas as pd
from transformers import AutoTokenizer

In [39]:
# LLaMA-3 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

In [80]:
#전처리완
preprocessed_file_path = r'data\preprocessed.json'
with open(preprocessed_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.head()

,source,title,content
0,경기도외국인지원센터_상담사례,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키스탄\n체류자격: E-9\n...
1,경기도외국인지원센터_상담사례,국민연금 반환일시금 신청 지원,상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 2023년 8...
2,경기도외국인지원센터_상담사례,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,상담유형: 체류자격\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 법무부 장...
3,경기도외국인지원센터_상담사례,단체행동을 통해 체불임금 해결한 사례,상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\n체류자격: E-9\n상담내...
4,경기도외국인지원센터_상담사례,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,상담유형: 고용허가\n거주지역: 시흥시\n국적: 필리핀\n체류자격: E-9\n상담내...


In [81]:
import pandas as pd
import random

def get_samples_by_token_length(df,israndom=True):
    # 0부터 9 사이의 랜덤한 정수를 생성
    if israndom == True:
        number = random.randint(0, 2048)
    else :
        number = 1
    # 토큰 길이 계산
    df['token_length'] = df['content'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=False)))
    
    # 각 범위에 해당하는 데이터 추출
    sample_512 = df[df['token_length'] <= 512].sample(n=1, random_state=number)
    sample_1024 = df[(df['token_length'] > 512) & (df['token_length'] <= 1024)].sample(n=1, random_state=number)
    sample_2048 = df[(df['token_length'] > 1024) & (df['token_length'] <= 2048)].sample(n=1, random_state=number)
    sample_2048_plus = df[df['token_length'] > 2048].sample(n=1, random_state=number)
    
    # 결과를 딕셔너리로 반환
    results = {
        "512토큰 이하": sample_512['content'].values[0] if not sample_512.empty else None,
        "513-1024토큰": sample_1024['content'].values[0] if not sample_1024.empty else None,
        "1025-2048토큰": sample_2048['content'].values[0] if not sample_2048.empty else None,
        "2048토큰 초과": sample_2048_plus['content'].values[0] if not sample_2048_plus.empty else None
    }
    
    return results

# 사용 예시
df = df[df['source']=='경기도외국인지원센터_상담사례']
samples = get_samples_by_token_length(df)

# 결과 출력
for category, content in samples.items():
    if content:
        print(f"{category}:")
        print(content[:100] + "..." if len(content) > 100 else content)  # 긴 내용은 앞부분만 출력
        print("\n")
    else:
        print(f"{category}: 해당 길이의 데이터가 없습니다.\n")

512토큰 이하:
상담유형: 임금체불
거주지역: 경기 파주시
국적: 태국
체류자격: E-9
상담내용: 2017년 입국하여 가구공장에서 계속 일을 하였음. 공장이 어려워졌다며 사업주가 사업장변경을 해...


513-1024토큰:
상담유형: 임금체불
거주지역: 경기 포천시
국적: 중국

상담내용: 중국 국적의 노동자 C는 특정활동(E-7) 자격으로 입국 후 포천시 소재 A음식점에 근로. 2020년 11월 사...


1025-2048토큰:
상담유형: 산재
거주지역: 경기 안산시
국적: 필리핀
체류자격: 기타
상담내용: 누적된 과로로 피를 토하고 쓰러진 이주근로자를 병원응급실로 이송하여 응급조치와 검사 후 중환자실(I...


2048토큰 초과:
상담유형: 부당해고
거주지역: 인천
국적: 캄보디아
체류자격: E-9
상담내용: 캄보디아 출신 M과 N(E-9)은 3년의 근로계약을 체결하고 인천의 문구제조 사업장에 입사하여 일하...




In [82]:
template_text1 = """
다음은 외국인 근로자의 상담 사례입니다. 이 사례를 외국인들이 쉽게 이해할 수 있도록 700자 이내로 요약해주세요. 요약 시 다음 사항을 지켜주세요:

쉬운 단어와 간단한 문장 구조를 사용하세요.
복잡한 법률 용어나 행정 용어는 풀어서 설명해주세요.
주요 정보는 유지하되, 불필요한 세부사항은 생략하세요.
시간 순서대로 사건을 정리하여 이해하기 쉽게 해주세요.
날짜는 표시하지 말아주세요.

외국인 근로자들이 비슷한 상황에 처했을 때 참고할 수 있도록 유용한 정보를 제공해주세요.

요약 예시 :
1. 태국 국적의 미등록 근로자가 경기 안산시에서 산업재해로 손가락을 상실한 사고를 당했습니다. 사업장은 1,000만원의 보상금을 제안했지만 근로자는 적정한 보상과 산재처리 가능 여부를 확인하고 싶어서 상담을 요청했습니다. 상담센터는 산재보상 절차를 설명하고 근로자의 의사를 확인한 후 산재신청서를 작성하여 사업주와 협의했습니다. 근로자는 현재 산재요양 중이며, 산재보상금 수령 후에도 사업주를 상대로 손해배상청구소송이 가능합니다. 센터는 근로자의 권리를 보장하고 최종 선택은 근로자가 할 수 있도록 지원하고 있습니다. 외국인 지원센터는 근로자가 적정한 보상을 받고 귀국할 수 있도록 도와주고 있습니다.
2. 베트남 국적 노동자 N씨가 파주시 H사업장에서 근로 후 4~6월 급여 체불 문제로 상담을 요청했습니다. 사업주는 지급을 거부하여 고용노동지청에 진정하였고, 결국 사업주와 분할 지급 계획을 합의하여 문제 해결되었습니다. 최종적으로 각각 260만원씩 지급받아 상담이 종결되었습니다. 외국인 근로자들은 고용노동지청에 도움을 요청하여 임금체불 문제를 해결할 수 있습니다.

상담사례 :
{text}

"""

In [46]:
template_text2 = """
다음은 외국인 근로자의 상담 사례입니다. 이 사례를 외국인들이 쉽게 이해할 수 있도록 700자 이내로 요약해주세요. 요약 시 다음 사항을 지켜주세요:

쉬운 단어와 간단한 문장 구조를 사용하세요.
복잡한 법률 용어나 행정 용어는 풀어서 설명해주세요.
주요 정보는 유지하되, 불필요한 세부사항은 생략하세요.
시간 순서대로 사건을 정리하여 이해하기 쉽게 해주세요.

요약에 포함해야 할 중요 정보:

근로자의 국적과 비자 종류
주요 문제점
문제 해결을 위해 한 일들
중요한 날짜들
최종 결과

외국인 근로자들이 비슷한 상황에 처했을 때 참고할 수 있도록 유용한 정보를 제공해주세요.

상담사례 :
{text}

"""

In [59]:
template_text3 = """
다음은 외국인 근로자의 상담 사례입니다. 이 사례를 외국인들이 쉽게 이해할 수 있도록 700자 이내로 요약해주세요. 요약 시 다음 사항을 지켜주세요:

요약에 포함해야 할 중요 정보:

근로자의 국적과 비자 종류
주요 문제점
문제 해결을 위해 한 일들
중요한 날짜들
최종 결과

외국인 근로자들이 비슷한 상황에 처했을 때 참고할 수 있도록 유용한 정보를 제공해주세요.
출력 시 요약문만 출력해주세요.

상담사례 :
{text}

"""

In [83]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

#llm 설정
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0,  # 창의성 (0.0 ~ 2.0)
                 max_tokens=2048,  # 최대 토큰수
                 openai_api_key="sk-None-DkSR8Er2hFvKWGZdF2hYT3BlbkFJ7xNVywsF2q1dxqrVq7zr")

# 템플릿 설정
prompt_template = PromptTemplate.from_template(template_text1)


#데이터 샘플 뽑기
samples = get_samples_by_token_length(df,israndom=True)

# 결과 출력
for category, content in samples.items():
    if content:
        
        print(f"###{category}({len(content)}글자)###:")
        # 템플릿에 값을 채워서 프롬프트를 완성
        filled_prompt = prompt_template.format(text= content)  
        output = llm.invoke(input = filled_prompt)
        
        print(f"원본 \n{content}")  # 긴 내용은 앞부분만 출력
        print("\n")
        print(f"요약 \n{output.content}")
        print("\n")

    else:
        print(f"{category}: 해당 길이의 데이터가 없습니다.\n")

###512토큰 이하(584글자)###:
원본 
상담유형: 고용허가
거주지역: 경기 안산시
국적: 필리핀
체류자격: E-9
상담내용: 고나는 김포에서 침대를 만드는 회사에서 일했습니다. 처음에는 필리핀 동료 근로자가 있었지만 몇 달 전에 그만두고 회사에는 모두 태국출신 이주노동자들만 남았습니다. 태국 노동자들과 대화도 안 통할뿐만 아니라 어렵고 힘든 일이 생기면 필리핀 출신인 자신에게만 시켜서 태국 근로자들과 사이가 좋지 않아서 회사를 이동하고 싶어함
진행 과정 및 결과: 7 월 9일
센터 방문하여 상담 접수

7 월 11일
사업주와 통화. 고나가 회사이동을 사업주에게 요청하던 중에 언어 차이로 오해도 쌓이고, 회사에 일하러 가지도 않아서 사업주가 화가 많이 남. 만약 고나가 내일까지 일을 하러 오지 않는다면 이탈신고를 할 계획이었다고 함

7 월 12일
사업주가 안산지역에 출장 나올 일이 있어서 와동에 있는 카페에서 본 활동가, 고나가 함께 만남. 쌓인 오해를 풀고, 고나도 한 달 동안 회사에 출근하면서 다른 사람을 찾을 때까지 출근을 한 이후 사업장 변경을 하기로 약속함 

7 월 26일
태국 출신 이주노동자에게 인수인계 해주고 사업장 변경신청된 것 확인
(사)안산이주민센터


요약 
요약:
고나는 필리핀 국적으로 경기 안산시에서 근무 중인데, 태국 출신 동료들과 의사소통이 어려워 회사 이동을 원함. 센터를 통해 사업주와의 오해를 해소하고 사업장 변경을 약속받아 문제 해결됨. 외국인 근로자는 센터를 통해 어려움을 해결할 수 있음.


###513-1024토큰(906글자)###:
원본 
상담유형: 기타
거주지역: 시흥시
국적: 필리핀
체류자격: E-9
상담내용: 사례자는 필리핀에서 아버님이 돌아가셔서 경조 휴가 관련 회사와의 중재를 요청해 옴에 따라 회사와 중재하여 10일의 휴가를 받아 주었으나 회사가 바쁘다는 것을 안다면 1주일만 필리핀에 휴가 가서 아버님 장례를 잘 치르고 귀국하여 회사에 복귀하여 잘 적응하고 있는 사례임.
진행 과정 및 결과: 
 (내방

자동생성

In [105]:
df.shape

(1700, 5)

In [104]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from tqdm import tqdm

# tqdm을 pandas와 함께 사용하기 위한 설정
tqdm.pandas()

#llm 설정
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0,  # 창의성 (0.0 ~ 2.0)
                 max_tokens=2048,  # 최대 토큰수
                 openai_api_key="sk-None-DkSR8Er2hFvKWGZdF2hYT3BlbkFJ7xNVywsF2q1dxqrVq7zr")

# 템플릿 설정
prompt_template = PromptTemplate.from_template(template_text1)

# 요약 함수 정의
def summarize_text(text):
    filled_prompt = prompt_template.format(text=text)
    output = llm.invoke(filled_prompt)
    return output.content

# DataFrame의 각 행에 대해 처리
def process_row(row):
    if row['token_length'] >= 512:
        return summarize_text(row['content'])
    else:
        return row['content']
    
# 전체 행 수 출력
print(f"총 처리할 행 수: {len(df)}")

# 'summary' 열 생성
# 주석처리
#df['summary'] = df.progress_apply(process_row, axis=1)

print(df[['content', 'token_length', 'summary']])

                                                content  token_length  \
0     상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키스탄\n체류자격: E-9\n...           913   
1     상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 2023년 8...           471   
2     상담유형: 체류자격\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 법무부 장...           547   
3     상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\n체류자격: E-9\n상담내...           441   
4     상담유형: 고용허가\n거주지역: 시흥시\n국적: 필리핀\n체류자격: E-9\n상담내...           692   
...                                                 ...           ...   
1695  상담유형: 임금체불\n거주지역: 경기 화성시\n국적: 태국\n체류자격: 기타\n상담...          1440   
1696  상담유형: 임금체불\n거주지역: 경기 여주시\n국적: 캄보디아\n체류자격: E-9\...          1469   
1697  상담유형: 임금체불\n거주지역: 경기 군포시\n국적: 스리랑카\n체류자격: E-9\...          1179   
1698  상담유형: 임금체불\n거주지역: 경기 안산시\n국적: 중국\n체류자격: H-2\n상...          1895   
1699  상담유형: 임금체불\n거주지역: 경기 안산시 단원구\n국적: 우즈베키스탄\n체류자격...          1507   

                                                summary  
0     우즈베키스탄 국적의 E-9 체류자인 외국인 근로자가 중도 퇴직 후 추가 납부세액 문...  
1     상

In [106]:
df.head()

,source,title,content,token_length,summary
0,경기도외국인지원센터_상담사례,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,상담유형: 체류자격\n거주지역: 여주시\n국적: 우즈베키스탄\n체류자격: E-9\n...,913,우즈베키스탄 국적의 E-9 체류자인 외국인 근로자가 중도 퇴직 후 추가 납부세액 문...
1,경기도외국인지원센터_상담사례,국민연금 반환일시금 신청 지원,상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 2023년 8...,471,상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 2023년 8...
2,경기도외국인지원센터_상담사례,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,상담유형: 체류자격\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 법무부 장...,547,필리핀 국적의 아동이 장기 미등록체류로 법무부에 구제대책을 요청했습니다. 부모의 유...
3,경기도외국인지원센터_상담사례,단체행동을 통해 체불임금 해결한 사례,상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\n체류자격: E-9\n상담내...,441,상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\n체류자격: E-9\n상담내...
4,경기도외국인지원센터_상담사례,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,상담유형: 고용허가\n거주지역: 시흥시\n국적: 필리핀\n체류자격: E-9\n상담내...,692,한국에 체류 중인 필리핀 국적 E-9 비자 소지자가 취업 어려움으로 쉼터에 입소하고...


In [107]:
# 토큰 길이 계산
df['summary_token_length'] = df['summary'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=False)))

In [110]:
df['token_length'].describe()

count    1700.000000
mean      568.723529
std       378.214810
min       119.000000
25%       345.000000
50%       460.500000
75%       653.250000
max      3813.000000
Name: token_length, dtype: float64

In [109]:
df['summary_token_length'].describe()

count    1700.000000
mean      277.813529
std       116.427446
min        89.000000
25%       174.000000
50%       250.500000
75%       380.000000
max       511.000000
Name: summary_token_length, dtype: float64

In [112]:
# 'source', 'title', 'summary' 열만 선택하고 'summary'를 'content'로 이름 변경
df_selected = df[['source', 'title', 'summary']].rename(columns={'summary': 'content'})

# 결과 확인
df_selected.head()

,source,title,content
0,경기도외국인지원센터_상담사례,중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원,우즈베키스탄 국적의 E-9 체류자인 외국인 근로자가 중도 퇴직 후 추가 납부세액 문...
1,경기도외국인지원센터_상담사례,국민연금 반환일시금 신청 지원,상담유형: 기타\n거주지역: 남양주시\n국적: 필리핀\n\n상담내용: 2023년 8...
2,경기도외국인지원센터_상담사례,외국인아동 교육권 보장을 위한 체류자격 부여자(G-1-85) 외국인등록 절차 지원,필리핀 국적의 아동이 장기 미등록체류로 법무부에 구제대책을 요청했습니다. 부모의 유...
3,경기도외국인지원센터_상담사례,단체행동을 통해 체불임금 해결한 사례,상담유형: 임금체불\n거주지역: 안산시\n국적: 필리핀\n체류자격: E-9\n상담내...
4,경기도외국인지원센터_상담사례,취업가능기간이 짧게 남아 취업이 어려운 노동자 구직활동 지원,한국에 체류 중인 필리핀 국적 E-9 비자 소지자가 취업 어려움으로 쉼터에 입소하고...


In [113]:
df_selected.shape

(1700, 3)

In [114]:
# DataFrame을 JSON으로 변환
json_data = df_selected.to_dict(orient='records')
json_data[0]

{'source': '경기도외국인지원센터_상담사례',
 'title': '중도 퇴사 후 근로소득 신고되지 않아 고용허가연장 안된 노동자 지원',
 'content': '우즈베키스탄 국적의 E-9 체류자인 외국인 근로자가 중도 퇴직 후 추가 납부세액 문제로 상담을 요청했습니다. 고용노동부에서 세금 체납으로 고용허가 연장이 불가능하다는 안내를 받았고, 이를 해결하기 위해 여주세무서와 회사 담당자와 함께 세금 문제를 상담하였습니다. 연말정산을 통해 환급을 받았고, 세금을 납부하여 고용허가 연장을 위해 이천세무서에 방문하였습니다. 세무서에서 2020년 세금 체납 사유를 설명하고, 체류자격 연장을 위한 절차를 안내받았습니다. 출입국사무소에서 체류자격 연장을 신청하고, 사회통합프로그램 사전평가 신청도 도와주었습니다. 이를 통해 외국인 근로자의 체류자격 연장 문제가 해결되었습니다.'}

In [1]:
import json
import pandas as pd

# 1. 기존 merged.json 파일 로드
with open('data\merged.json', 'r', encoding='utf-8') as file:
    existing_data = json.load(file)

# 2. '경기도 외국인 상담사례'가 아닌 항목만 필터링
filtered_data = [item for item in existing_data if item['source'] != '경기도외국인지원센터_상담사례']

# 3. 새로운 데이터프레임 로드 (이전 단계에서 생성한 df_selected를 사용한다고 가정)
# df_selected가 이미 메모리에 있다고 가정합니다. 없다면 아래 줄의 주석을 해제하고 파일에서 로드하세요.
# df_selected = pd.read_csv('your_new_data.csv')

# 4. 데이터프레임을 딕셔너리 리스트로 변환
new_data = df_selected.to_dict(orient='records')

# 5. 필터링된 기존 데이터와 새 데이터 합치기
merged_data = filtered_data + new_data

# 6. 결과를 새 JSON 파일로 저장
with open('merged_add_summary.json', 'w', encoding='utf-8') as file:
    json.dump(merged_data, file, ensure_ascii=False, indent=2)

print(f"총 {len(merged_data)}개의 항목이 merged_add_summary.json 파일에 저장되었습니다.")

NameError: name 'df_selected' is not defined

In [4]:
with open('data\merged_add_summary.json', 'r', encoding='utf-8') as file:
    existing_data = json.load(file)

selected_data = [item for item in existing_data if item['source'] == '경기도외국인지원센터_상담사례']

len(selected_data)

1700

In [5]:
import json
import pandas as pd

# 1. 기존 merged.json 파일 로드
with open('data\merged.json', 'r', encoding='utf-8') as file:
    existing_data = json.load(file)

# 2. '법무부 외국인 체류관리'가 아닌 항목만 필터링
filtered_data = [item for item in existing_data if item['source'] != '경기도외국인지원센터_상담사례']

# 3. 새로운 데이터프레임 로드 (이전 단계에서 생성한 df_selected를 사용한다고 가정)
# df_selected가 이미 메모리에 있다고 가정합니다. 없다면 아래 줄의 주석을 해제하고 파일에서 로드하세요.
# df_selected = pd.read_csv('your_new_data.csv')

# 4. 데이터프레임을 딕셔너리 리스트로 변환
#new_data = df_selected.to_dict(orient='records')

# 5. 필터링된 기존 데이터와 새 데이터 합치기
merged_data = filtered_data + selected_data

# 6. 결과를 새 JSON 파일로 저장
with open('merged_add_summary.json', 'w', encoding='utf-8') as file:
    json.dump(merged_data, file, ensure_ascii=False, indent=2)

print(f"총 {len(merged_data)}개의 항목이 merged_add_summary.json 파일에 저장되었습니다.")

총 3433개의 항목이 merged_add_summary.json 파일에 저장되었습니다.


### context

In [36]:
# ChatOpenAI 인스턴스를 생성하여 프롬프트 텍스트를 전달하고, 모델의 출력을 StrOutputParser를 통해 문자열로 변환하는 LLM 체인을 구성합니다.
# invoke 메소드를 사용하여 파이프라인을 실행하고, 최종적으로 문자열 출력을 얻습니다. 모델의 응답은 프롬프트에 주어진 문장을 영어로 번역한 텍스트가 출력됩니다.

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0,  # 창의성 (0.0 ~ 2.0)
                 max_tokens=1024,  # 최대 토큰수
                 openai_api_key="sk-None-DkSR8Er2hFvKWGZdF2hYT3BlbkFJ7xNVywsF2q1dxqrVq7zr")


#chain = filled_prompt | llm | StrOutputParser()
output = llm.invoke(input = filled_prompt)


In [37]:
context = output.content
context

'요약:\n한국에 미등록체류 중인 필리핀 국적 노동자가 강제출국되면서 국민연금 반환일시금을 받지 못한 문제가 발생. 본국에서 받을 방법이 없어 도움 요청. 관련 서류를 구비하고 대사관 공증을 받아 국민연금 관리공단에 제출. 소멸시효 안내 후 서류 제출 완료. 해외에 있는 노동자에게 직접 통화하여 반환일시금 지급 예정. 국민연금 반환일시금 접수 완료. (500자)'

In [28]:
context = output.content

combined_prompt = (
    PromptTemplate.from_template("이 시스템은 외국인 근로자들을 상담해주는 시스템입니다. \n\n{context}를 참고해 대화 해주세요.")
    + "user: {user_input}"
    )
print(combined_prompt)

chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"context": context, "user_input":"나도 출입국사무소에서 연장 신청을 해야겠네?"})

input_variables=['context', 'user_input'] template='이 시스템은 외국인 근로자들을 상담해주는 시스템입니다. \n\n{context}를 참고해 대화 해주세요.user: {user_input}'


'시스템: 네, 출입국사무소에서 체류자격 연장을 신청하시는 것이 좋습니다. 그 전에 세금 체납 문제나 다른 이슈가 있는지 확인하고, 필요한 서류를 준비하여 방문하시기 바랍니다. 또한, 법무부 결과에 따라 소송 진행이 가능하니 상황에 맞게 대응하시기 바랍니다. 부디 원활한 체류자격 연장을 위해 노력해주시기 바랍니다.'

In [25]:
# from langchain.llms import OpenAI
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationBufferMemory

# # OpenAI API 키 설정 (환경 변수로 설정하는 것이 좋습니다)
# #import os
# #os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# # LLM 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
#                  temperature=0,  # 창의성 (0.0 ~ 2.0)
#                  max_tokens=2048,  # 최대 토큰수
#                  openai_api_key="sk-None-DkSR8Er2hFvKWGZdF2hYT3BlbkFJ7xNVywsF2q1dxqrVq7zr")

# # 대화 메모리 초기화
# memory = ConversationBufferMemory()

# template = """이 시스템은 외국인 근로자들을 상담해주는 시스템입니다.
# 답변은 친절하고 이해하기 쉬운 언어로 제공해주세요.
# 전문 용어는 가능한 쉽게 풀어서 설명해주세요.

# 현재 대화:
# {history}
# Human: {input}
# AI Assistant:"""

# prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# # 대화 체인 생성
# conversation = ConversationChain(
#     llm=llm, 
#     memory=memory,
#     verbose=True,
#     prompt = prompt
# )

# # 대화 시작
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ['quit', 'exit', 'bye']:
#         print("AI: Goodbye!")
#         break
    
#     response = conversation.predict(input=user_input)
#     print(f"AI: {response}")